# Загрузка конфигурации

In [1]:
import math
import os

from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from scipy.sparse import csr_matrix

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

# Загрузка данных

In [2]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')
interactions_df = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

In [3]:
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

In [4]:
interactions_df.shape

(263874, 5)

# Подготовка данных

In [5]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [6]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
                                                            how = 'right',
                                                            left_on = 'user_id',
                                                            right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


In [7]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


In [8]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                                               stratify=interactions_full_df['user_id'], 
                                                               test_size=0.20,
                                                               random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [9]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [11]:
# Constants
SEED = 42 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.05 # lambda for L2 reg. ()
NUM_EPOCHS = 125 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 2048 # training batch size
EVAL_BATCH_SIZE = 2048 # evaluation batch size.
DEVICE = 'cuda' #'cuda' # device to make the calculations on

In [12]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

C:\Users\BIT\AppData\Local\Temp\ipykernel_2204\627896225.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())


In [13]:
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [16]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__() # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [17]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

# Модель

In [18]:
class Model(nn.Module):
    def __init__(self, in_and_out_features = 8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size = 1000

        self.sequential = nn.Sequential( # NN architecure, where the modules modify the data sequentially
            nn.Linear(in_and_out_features, self.hidden_size), # Linear transformation in_and_out_features -> hidden_size
            nn.ReLU(), # Activation function
            nn.Linear(self.hidden_size, self.hidden_size//2), # Linear transformation hidden_size -> hidden_size / 2
            nn.ReLU(), # Activation function
            nn.Linear(self.hidden_size//2, self.hidden_size//4), # Linear transformation hidden_size / 2 -> hidden_size / 4
            nn.ReLU(), # Activation function
            nn.Linear(self.hidden_size//4, self.hidden_size//2), # Linear transformation hidden_size / 4 -> hidden_size / 2
            nn.ReLU(), # Activation function
            nn.Linear(self.hidden_size//2, self.hidden_size), # Activation function hidden_size / 2 -> hidden_size
            nn.ReLU(), # Activation function
            nn.Linear(self.hidden_size, in_and_out_features) # Another Linear transformation hidden_size -> in_and_out_features
        )

    def forward(self, x): # In the forward function, you define how your model runs, from input to output 
        x = self.sequential(x)
        return x

In [19]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [20]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0 
            for batch in dls[stage]:
                batch = batch.to(DEVICE)
                x_pred = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            
    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.231827,1.978129
1,1,2.015750,2.134739
2,2,1.888301,2.095499
3,3,1.751246,1.977681
4,4,1.630306,1.851590
...,...,...,...
120,120,0.726853,1.373316
121,121,0.722432,1.352404
122,122,0.713016,1.348140
123,123,0.721052,1.347090


In [22]:
# torch.save(model, 'AE.pt')

In [23]:
model = torch.load('AE.pt')

In [24]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test).to(DEVICE))
X_pred

tensor([[ 5.6592,  7.1618, -0.3002,  ..., -0.7707, -0.3882, -0.1800],
        [ 2.4231,  3.1517,  1.2136,  ..., -0.3672,  0.1970, -0.6836],
        [ 4.9495,  4.5450,  3.7632,  ..., -0.5118, -0.0138, -0.1757],
        ...,
        [ 2.6256,  3.5006,  1.4880,  ..., -0.4783,  0.2624, -0.5053],
        [ 4.0173,  5.4282,  3.5894,  ..., -0.6349,  0.4016, -0.9525],
        [ 4.8283,  5.3463,  4.2885,  ..., -0.4425, -0.0335,  0.0667]],
       device='cuda:0')

In [25]:
class AERecommender:
    
    MODEL_NAME = 'Autoencoder'
    
    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)

        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}
    
ae_recommender_model = AERecommender(X_pred, X_train, X_train)

In [26]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.24911503993290882, 'recall@10': 0.5241596536222415}